In [62]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os
from tqdm import tqdm_notebook as tqdm

In [63]:
path = '.\\valid\\'

Считываем данные по трекам с привязанными остановками

In [64]:
res_stop = pd.read_csv('res2020_09_30.csv')

In [65]:
res_stop=res_stop.rename(columns={'MODULEID':'tabelnum'})

Определяем времена начала и конца для каждого табельного номера

In [66]:
mr_id_marsh_valid_id = pd.read_csv('./data/tol_marsh_PATP.csv', sep = ';', encoding = 'windows-1251')

In [67]:
mr_id_marsh_valid_id.head()

,Код Маршрута,Код предприятия,Название маршрута,Вид транспорта,Название,mr_id
0,51003456789,25,21 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,21,2227623072
1,52103456789,25,40 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,40,2227623079
2,51103456789,25,22 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,22,2227623045
3,52603456789,25,73 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,73,2227622952
4,52303456789,25,46 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,46,2227623084


In [68]:
validation= pd.read_csv(path+'2020.09.29.csv', sep = ',', encoding = 'cp1251')
validation=validation.merge(mr_id_marsh_valid_id,on =['Код Маршрута'])
validation=validation[validation['mr_id'].isin(res_stop['mr_id'].unique())]
validation['time']=pd.to_datetime(validation['Дата и время поездки'])
validation['hour']=validation['time'].dt.hour
validation.loc[validation.hour ==0, 'hour'] = 24
validation.loc[validation.hour ==1, 'hour'] = 25
validation.loc[validation.hour ==2, 'hour'] = 26
validation.loc[validation.hour ==3, 'hour'] = 27
validation['min']=validation['time'].dt.minute
validation['sec']=validation['time'].dt.second
validation['time']=(validation['hour'].astype(int))*3600+(validation['min'].astype(int))*60+(validation['sec'].astype(int))
validation=validation.drop(['hour'], axis=1).drop(['min'], axis=1).drop(['sec'], axis=1)
validation_start=pd.DataFrame(validation.groupby(['Код Маршрута','Табельный номер']).agg({'time': np.min})).reset_index().rename(columns={"time": "start"})
validation_finish=pd.DataFrame(validation.groupby(['Код Маршрута','Табельный номер']).agg({'time': np.max})).reset_index().rename(columns={"time": "finish"})
validation_start_finish=validation_start.merge(validation_finish, on=['Код Маршрута','Табельный номер'])
validation2=validation.merge(validation_start_finish, on=['Код Маршрута','Табельный номер'])
del validation_start
del validation_finish
del validation

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [69]:
df_4 = res_stop
df_4['time2']=pd.to_datetime(df_4['time'])
df_4['hour']=df_4['time2'].dt.hour
df_4['min']=df_4['time2'].dt.minute
df_4['sec']=df_4['time2'].dt.second
df_4.loc[df_4.hour ==0, 'hour'] = 24
df_4.loc[df_4.hour ==1, 'hour'] = 25
df_4.loc[df_4.hour ==2, 'hour'] = 26
df_4.loc[df_4.hour ==3, 'hour'] = 27
df_4['time2']=(df_4['hour'].astype(int))*3600+(df_4['min'].astype(int))*60+(df_4['sec'].astype(int))
#df_4=df_4.drop(['hour'], axis=1).drop(['min'], axis=1).drop(['sec'], axis=1)
df_4_start=pd.DataFrame(df_4.groupby(['mr_id','tabelnum']).agg({'time2': np.min})).reset_index().rename(columns={"time2": "start"})
df_4_finish=pd.DataFrame(df_4.groupby(['mr_id','tabelnum']).agg({'time2': np.max})).reset_index().rename(columns={"time2": "finish"})
df_4_start_finish=df_4_start.merge(df_4_finish, on=['mr_id','tabelnum']).sort_values(by=['mr_id', 'start'])
del df_4_start
del df_4_finish

In [70]:
validation_start_finish2=validation_start_finish.merge(mr_id_marsh_valid_id,on =['Код Маршрута'])
mrid=validation_start_finish2['mr_id'].unique()
df_4_start_finish=df_4_start_finish[df_4_start_finish['mr_id'].isin(mrid)]
validation_start_finish2=validation_start_finish2.sort_values(by=['mr_id', 'start'])
df_4_start_finish=df_4_start_finish.sort_values(by=['mr_id', 'start'])

In [71]:
validation_start_finish2.head()

,Код Маршрута,Табельный номер,start,finish,Код предприятия,Название маршрута,Вид транспорта,Название,mr_id
64,50203456789,2652,19618,28473,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934
65,50203456789,2687,21395,28836,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934
73,50203456789,31388,21507,62189,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934
74,50203456789,31897,21712,60729,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934
58,50203456789,31897,22046,29092,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934


In [72]:
df_4_start_finish_mem=df_4_start_finish

In [73]:
res=validation_start_finish2

In [74]:
df_4_start_finish

,mr_id,tabelnum,start,finish
0,2.227623e+09,8561,86400,86400
1,2.227623e+09,8568,86400,86400
2,2.227623e+09,8579,86400,86400
3,2.227623e+09,8582,86400,86400
4,2.227623e+09,8583,86400,86400
...,...,...,...,...
814,4.009117e+09,8690,86400,86400
815,4.009117e+09,8691,86400,86400
816,4.009117e+09,8693,86400,86400
817,4.009117e+09,8697,86400,86400


1. Раскидываем машины по табельным номерам валидаций. Жесткое условие (все валидации внутри трека)

In [75]:
k=0
res=validation_start_finish2
tabelnum = []
start = []
finish = []
tracks=pd.DataFrame()
for index, row in res.iterrows():
    stops=df_4_start_finish[(df_4_start_finish['mr_id']==row['mr_id'])
                    &(df_4_start_finish['start']<=row['start'])
                    &(df_4_start_finish['finish']>row['finish'])]
    df_4_start_finish=df_4_start_finish[~df_4_start_finish.isin(stops[0:1])].dropna(how = 'all')
    if stops.empty:
        tabelnum.append(0)
        start.append(0)
        finish.append(0)
    else:
        tabelnum_unic=stops['tabelnum'].values
        tabelnum.append(tabelnum_unic[0])
        start_unic=stops['start'].values
        start.append(start_unic[0])
        finish_unic=stops['finish'].values
        finish.append(finish_unic[0])
res['tabelnum']=tabelnum
res['start_track']=start
res['finish_track']=finish

In [76]:
res.head()

,Код Маршрута,Табельный номер,start,finish,Код предприятия,Название маршрута,Вид транспорта,Название,mr_id,tabelnum,start_track,finish_track
64,50203456789,2652,19618,28473,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934,0,0,0
65,50203456789,2687,21395,28836,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934,0,0,0
73,50203456789,31388,21507,62189,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934,0,0,0
74,50203456789,31897,21712,60729,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934,0,0,0
58,50203456789,31897,22046,29092,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934,0,0,0


In [77]:
len(df_4_start_finish)

799

2. Раскидываем машины по табельным номерам валидаций. Мягкое условие 1 (все валидации после начала трека)

In [78]:
k=0
tabelnum = []
start = []
finish = []
stops=pd.DataFrame()
for index, row in res.iterrows():
    if row['tabelnum']==0:
        stops=df_4_start_finish[(df_4_start_finish['mr_id']==row['mr_id'])
                    &(df_4_start_finish['start']<=row['start'])]
        df_4_start_finish=df_4_start_finish[~df_4_start_finish.isin(stops[0:1])].dropna(how = 'all')
        if stops.empty:
            tabelnum.append(0)
            start.append(0)
            finish.append(0)
        else:
            tabelnum_unic=stops['tabelnum'].values
            tabelnum.append(tabelnum_unic[0])
            start_unic=stops['start'].values
            start.append(start_unic[0])
            finish_unic=stops['finish'].values
            finish.append(finish_unic[0])
    else:
        tabelnum.append(0)
        start.append(0)
        finish.append(0)        
res['tabelnum_2']=tabelnum
res['start_track_2']=start
res['finish_track_2']=finish

3. Раскидываем машины по табельным номерам валидаций. Мягкое условие 2 (все валидации до конца трека)

In [79]:
k=0
tabelnum = []
start = []
finish = []
stops=pd.DataFrame()
for index, row in res.iterrows():
    if row['tabelnum']==0:
        stops=df_4_start_finish[(df_4_start_finish['mr_id']==row['mr_id'])
                    &(df_4_start_finish['finish']>row['finish'])]
        df_4_start_finish=df_4_start_finish[~df_4_start_finish.isin(stops[0:1])].dropna(how = 'all')
        if stops.empty:
            tabelnum.append(0)
            start.append(0)
            finish.append(0)
        else:
            tabelnum_unic=stops['tabelnum'].values
            tabelnum.append(tabelnum_unic[0])
            start_unic=stops['start'].values
            start.append(start_unic[0])
            finish_unic=stops['finish'].values
            finish.append(finish_unic[0])
    else:
        tabelnum.append(0)
        start.append(0)
        finish.append(0)        
res['tabelnum_3']=tabelnum
res['start_track_3']=start
res['finish_track_3']=finish

In [80]:
res.head()

,Код Маршрута,Табельный номер,start,finish,Код предприятия,Название маршрута,Вид транспорта,Название,mr_id,tabelnum,start_track,finish_track,tabelnum_2,start_track_2,finish_track_2,tabelnum_3,start_track_3,finish_track_3
64,50203456789,2652,19618,28473,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934,0,0,0,0.0,0.0,0.0,8568.0,86400.0,86400.0
65,50203456789,2687,21395,28836,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934,0,0,0,0.0,0.0,0.0,8579.0,86400.0,86400.0
73,50203456789,31388,21507,62189,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934,0,0,0,0.0,0.0,0.0,8582.0,86400.0,86400.0
74,50203456789,31897,21712,60729,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934,0,0,0,0.0,0.0,0.0,8583.0,86400.0,86400.0
58,50203456789,31897,22046,29092,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934,0,0,0,0.0,0.0,0.0,8606.0,86400.0,86400.0


In [81]:
len(df_4_start_finish)

490

4.Дозаполняем табельные номера тем, что не вошло

In [82]:
k=0
df_4_start_finish=df_4_start_finish_mem
tabelnum = []
start = []
finish = []
stops=pd.DataFrame()
for index, row in res.iterrows():
    if row['tabelnum']==0:
        stops=df_4_start_finish[(df_4_start_finish['mr_id']==row['mr_id'])
                    &(df_4_start_finish['finish']>row['finish'])]
        if stops.empty:
            tabelnum.append(0)
            start.append(0)
            finish.append(0)
        else:
            tabelnum_unic=stops['tabelnum'].values
            tabelnum.append(tabelnum_unic[0])
            start_unic=stops['start'].values
            start.append(start_unic[0])
            finish_unic=stops['finish'].values
            finish.append(finish_unic[0])
    else:
        tabelnum.append(0)
        start.append(0)
        finish.append(0)        
res['tabelnum_4']=tabelnum
res['start_track_4']=start
res['finish_track_4']=finish

In [83]:
res.to_csv(path+'for_mege_tabelnum_2020_09_30.csv')
#df_4_start_finish.to_csv(path+'for_mege_tabelnum_ost_29.csv')

In [84]:
len(res[res['tabelnum_3']>0])

297

In [85]:
len(res[res['tabelnum']==0])

443

Перезаписываем данные в столбцах

In [86]:
res.loc[res.tabelnum==0, 'tabelnum'] = res.loc[res.tabelnum==0, 'tabelnum_3']
res.loc[res.tabelnum==0, 'tabelnum'] = res.loc[res.tabelnum==0, 'tabelnum_2']
res.loc[res.start_track==0, 'start_track'] = res.loc[res.start_track==0, 'start_track_2']
res.loc[res.start_track==0, 'start_track'] = res.loc[res.start_track==0, 'start_track_3']
res.loc[res.finish_track==0, 'finish_track'] = res.loc[res.finish_track==0, 'finish_track_2']
res.loc[res.finish_track==0, 'finish_track'] = res.loc[res.finish_track==0, 'finish_track_3']

In [87]:
res.loc[res.tabelnum==0, 'tabelnum'] = res.loc[res.tabelnum==0, 'tabelnum_4']
res.loc[res.start_track==0, 'start_track'] = res.loc[res.start_track==0, 'start_track_4']
res.loc[res.finish_track==0, 'finish_track'] = res.loc[res.finish_track==0, 'finish_track_4']

Пересохранили табельный номер как int, чтобы правильнее можно было merge делать

In [88]:
table_num_merge_29=res[['Табельный номер','mr_id','tabelnum']]

In [89]:
len(validation2['mr_id'].unique())

39

In [90]:
table_num_merge_29[['Табельный номер','tabelnum']]

,Табельный номер,tabelnum
64,2652,8568.0
65,2687,8579.0
73,31388,8582.0
74,31897,8583.0
58,31897,8606.0
...,...,...
432,33595,8582.0
406,14555,8561.0
408,14555,8568.0
405,14478,8579.0


In [91]:
table_num_merge_29.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 443 entries, 64 to 407
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Табельный номер  443 non-null    object 
 1   mr_id            443 non-null    int64  
 2   tabelnum         443 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 13.8+ KB


In [92]:
validation4=validation2.merge(table_num_merge_29,on =['mr_id','Табельный номер'],how='left')
validation4=validation4.sort_values(by=['mr_id','Табельный номер','time'])

In [93]:
validation4

,Дата и время поездки,Терминал,Номер билета,Номер кристалла,TCAT,NUM_TRIP,Код Маршрута,Табельный номер,Код предприятия_x,Код предприятия_y,Название маршрута,Вид транспорта,Название,mr_id,time,start,finish,tabelnum
8512,2020-09-29 07:51:13,81526131,8,E34CC6C5,СОЦ,65,50203456789,313,25,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934,28273,28273,28508,8703.0
8513,2020-09-29 07:52:55,81526131,16,BDC0CDAE,СОЦ,26,50203456789,313,25,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934,28375,28273,28508,8703.0
8514,2020-09-29 07:55:08,81526131,18,8E43CCFD,СОЦ,39,50203456789,313,25,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934,28508,28273,28508,8703.0
8451,2020-09-29 06:19:42,81526761,2,A3325D42,СОЦ,38,50203456789,888,25,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934,22782,22782,28836,8664.0
8452,2020-09-29 06:20:14,81526761,8,DDD9D2AE,СОЦ,76,50203456789,888,25,25,12 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,12,2227622934,22814,22782,28836,8664.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43366,2020-09-29 12:49:40,81526487,27,9EE002CD,СОЦ,34,71103456789,14478,25,25,62 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,62,4009117260,46180,45302,46180,8582.0
43356,2020-09-29 09:57:18,81526930,19,2568CD1F,ЕТК,0,71103456789,14555,25,25,62 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,62,4009117260,35838,35838,41522,8568.0
43357,2020-09-29 09:57:21,81526930,20,2568CD1F,ЕТК,0,71103456789,14555,25,25,62 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,62,4009117260,35841,35838,41522,8568.0
43358,2020-09-29 09:57:39,81526930,24,1BCF08BC,ЕТК,0,71103456789,14555,25,25,62 ГОРОДСКОЙ МАРШРУТ,АВТОБУС ГОРОД,62,4009117260,35859,35838,41522,8568.0


In [94]:
del validation2

Переводим столбец с временем в формат INT и прописываем для каждого табельного номера время начала и конца

In [95]:
res_stop

,mr_id,rl_id,tabelnum,rc_orderby,ssId,time,time_start,time_finish,ss_lat,ss_long,geometry,time2,hour,min,sec
0,2.227623e+09,2.227623e+09,8561,1.0,stop__9854427,115978,0,0.0,53.557965,49.228861,POINT (49.229126 53.558048),86400,24,0,0
1,2.227623e+09,2.227623e+09,8561,37.0,stop__9854249,167443,0,NaN,53.505270,49.409990,POINT (49.411068 53.505283),86400,24,0,0
2,2.227623e+09,2.227623e+09,8568,1.0,stop__9854427,144062,0,0.0,53.557965,49.228861,POINT (49.229126 53.55809),86400,24,0,0
3,2.227623e+09,2.227623e+09,8568,17.0,stop__9854292,150750,0,NaN,53.547322,49.354672,POINT (49.355515 53.547424),86400,24,0,0
4,2.227623e+09,2.227623e+09,8579,1.0,stop__9854427,143975,0,0.0,53.557965,49.228861,POINT (49.230267 53.557941),86400,24,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811,4.009117e+09,4.009119e+09,8690,31.0,1543174470,149351,0,NaN,53.502156,49.263104,POINT (49.262581 53.502277),86400,24,0,0
2812,4.009117e+09,4.009119e+09,8691,31.0,1543174470,169528,0,NaN,53.502156,49.263104,POINT (49.262196 53.502495),86400,24,0,0
2813,4.009117e+09,4.009119e+09,8693,31.0,1543174470,144534,0,NaN,53.502156,49.263104,POINT (49.262493 53.502247),86400,24,0,0
2814,4.009117e+09,4.009119e+09,8697,31.0,1543174470,171370,0,NaN,53.502156,49.263104,POINT (49.263096 53.501667),86400,24,0,0


In [96]:
validation4['tr_id']='';

In [97]:
validation4=validation4[validation4['mr_id'].isin(res_stop['mr_id'].unique())]

Смотрим сколько маршрутов у нас есть и в валидациях, и в треках

In [98]:
len(validation4['mr_id'].unique())

39

In [99]:
validation4['tabelnum'].unique()

array([8703., 8664., 8671., 8627., 8682., 8613., 8693., 8638., 8635.,
       8677., 8644., 8606.,    0., 8641., 8674., 8561., 8568., 8579.,
       8691., 8637., 8632., 8690., 8697., 8582., 8583., 8704., 8650.,
       8586., 8599., 8560.])

In [100]:
res_stop2=res_stop

In [101]:
k=0
res=validation4
result = []
result2 = []
stops=pd.DataFrame()
for index, row in validation4.iterrows():
    stops=res_stop2[(res_stop2['mr_id']==row['mr_id'])
                    &(res_stop2['tabelnum']==row['tabelnum'])
                    &(res_stop2['time_start']<=row['time'])
                    &((res_stop2['time_finish']>row['time'])|(res_stop2['time_finish'].isna()))]
    if stops.empty:
        result.append(0)
        result2.append(0)
    else:
        stop_unic=stops['ssId'].values
        result.append(stop_unic[0])
        rl_id=stops['rl_id'].values
        result2.append(rl_id[0])
res['ssId']=result
res['rl_id']=result2
FOR_MATRIX=res

In [102]:
len(res[res['ssId']==0])

1195

In [103]:
stops=pd.read_csv('./data/stop_seq_patp_for_28.csv', sep = ';', encoding='utf-8')

In [104]:
stops=stops[['ssId','ss_lat','ss_long']].drop_duplicates()

In [105]:
stops

,ssId,ss_lat,ss_long
0,stop__9854427,53.557965,49.228861
1,stop__9854425,53.557324,49.236213
2,stop__9854423,53.556677,49.243121
3,stop__9854421,53.556088,49.250009
4,stop__9854419,53.555488,49.256905
...,...,...,...
2689,stop__9854732,53.531213,49.554170
2690,stop__9854729,53.527459,49.549038
2691,stop__9854727,53.522507,49.541815
2692,stop__9854725,53.543047,49.513853


In [106]:
res2=res.merge(stops, on='ssId', how='left')

In [107]:
res2['c']=1

In [108]:
res2[res2['ssId']==0]['mr_id'].unique()

array([2227622934, 2227623001, 2227623004, 2227623036, 2227623045,
       2227623048, 2227623055, 2227623094], dtype=int64)

In [109]:
rl_orderby=res_stop[['ssId','rl_id','mr_id','rc_orderby']].drop_duplicates()

In [110]:
rl_orderby.head()

,ssId,rl_id,mr_id,rc_orderby
0,stop__9854427,2.227623e+09,2.227623e+09,1.0
1,stop__9854249,2.227623e+09,2.227623e+09,37.0
3,stop__9854292,2.227623e+09,2.227623e+09,17.0
6,stop__9854234,2.227623e+09,2.227623e+09,29.0
8,stop__9854417,2.227623e+09,2.227623e+09,6.0


In [111]:
len(res2)

43882

In [112]:
res2=res2.merge(rl_orderby, on=['ssId','rl_id','mr_id'], how='left')

In [114]:
len(res2[res2['ssId']==0])

1195

In [115]:
res2.to_csv(path+'For_matrix_2020_09_30.csv')

In [ ]:
marsh=pd.read_csv(path+'Маршруты.csv', sep = ';', encoding='utf-8')

In [ ]:
marsh.head()

In [ ]:
res2=res2.merge(marsh, on='Код Маршрута', how='left')

In [ ]:
res2.to_csv('stops_2020_09_29.csv')

In [ ]:
res2.head()

In [ ]:
for_karta3=res2.groupby(['ssId','ss_lat','ss_long']).agg({'c': np.sum}).reset_index()
for_karta3=for_karta3.groupby(['ssId','ss_lat','ss_long']).agg({'c': np.sum}).reset_index()
gdf = gpd.GeoDataFrame(
    for_karta3, geometry=gpd.points_from_xy(x=for_karta3.ss_long, y=for_karta3.ss_lat)
)
gdf.crs='epsg:4326'
gdf.to_file(fr"./res/stops_2020_09_29.json", index=False, driver="GeoJSON")